# Packages

In [1]:
using FinEtools, FinEtoolsAcoustics, LinearAlgebra, StatsBase, HDF5, Random
cd(desktopPath) # change directory

# Initial setup

In [2]:
rho = 1.21 * phun("kg/m^3") # mass density
c  = 343.0 * phun("m/s") # sound speed
bulk =  c ^ 2 * rho
frequency = 500.0 # frequency of the incident wave, Hz
omega = 2 * pi * frequency
dt = 1.0 / frequency / 15
tfinal = 140 * dt
nsteps = round(tfinal / dt) + 1
println("Step takes $(sciNotation(dt, 3)) s")
println("Total of $(round(Int, nsteps)) steps")
println("Final time: $(sciNotation(tfinal, 3)) s")

Step takes 1.333E-4 s
Total of 141 steps
Final time: 1.867E-2 s


# Mesh

Geometry

In [3]:
# list of meshes
Meshes = Array{Tuple{FENodeSet, AbstractFESet}, 1}()
# Q4 discretization of large rectangular domain divisions
push!(Meshes, Q4quadrilateral([0.0 0.0; 1.2 2.6], 10, 26)) # 1
push!(Meshes, Q4quadrilateral([0.0 2.6; 1.6 5.45], 13, 30)) # 2
push!(Meshes, Q4quadrilateral([1.35 0.0; 4.05 2.6], 25, 24)) # 3
push!(Meshes, Q4quadrilateral([1.6 2.6; 4.05 5.8], 23, 34)) # 4
push!(Meshes, Q4quadrilateral([4.25 1.2; 5.25 4.5], 10, 35)) # 5
push!(Meshes, Q4quadrilateral([4.05 4.5; 5.25 5.3], 12, 9)) # 6
push!(Meshes, Q4quadrilateral([4.25 5.3; 5.25 5.85], 10, 4)) # 7
push!(Meshes, Q4quadrilateral([5.25 1.2; 7.0 3.92], 22, 29)) # 8
# merge meshes
fens, outputfes = mergenmeshes(Meshes, 0.05)
# concatenate connectivities
fes = cat(outputfes[5], cat(outputfes[6], cat(outputfes[7], outputfes[8])))
fes = cat(outputfes[1], cat(outputfes[2], cat(outputfes[3], cat(outputfes[4], fes))))
println("$(size(fens.xyz, 1)) nodes")
println("$(size(fes.conn, 1)) elements")
println("$(round(Int, size(fens.xyz, 1) * nsteps)) dataset points (using Q4)")

3362 nodes
3168 elements
474042 dataset points (using Q4)


Subdomain selection

In [4]:
# Identify boundary finite element set
bfes = meshboundary(fes)
# In case there are any unconnected nodes, remove them, and renumber the elements.
connected = findunconnnodes(fens, fes)
fens, new_numbering = compactnodes(fens, connected)
fess = renumberconn!(fes, new_numbering)
# The geometry and the solution (pressure) fields
geom = NodalField(fens.xyz)
P = NodalField(zeros(FCplxFlt, size(fens.xyz, 1), 1));

Export domain

In [5]:
# Export three VTK files: one for the interior of the fluid, and one for the boundary
# vtkexportmesh("./vtkFiles/interior.vtk", fes.conn, fens.xyz, FinEtools.MeshExportModule.VTK.Q4);

# Discrete model

Setup

In [6]:
# Number the degrees of freedom in the pressure field.
numberdofs!(P)
# Create the finite element machine for the fluid.
material = MatAcoustFluid(bulk, rho)
femm  =  FEMMAcoust(IntegDomain(fes, GaussRule(3, 2)), material)
# Use the machine calculate the acoustic stiffness and mass matrices.
S  =  acousticstiffness(femm, geom, P)
C  =  acousticmass(femm, geom, P)
# "damping" matrix for the absorbing boundary conditions (ABCs)
D  =  acousticABC(
  FEMMAcoustSurf(IntegDomain(bfes, GaussRule(2, 2)), material), geom, P
)
# node IDs for load positions
loadNode = eachindex(fens) |> collect
# loadPos = subset(fes, selectelem(fens, fes, withnodes = [2049, 2050, 2075, 2074]))
# dipfemm = FEMMAcoustSurf(IntegDomain(loadNode, GaussRule(2, 2)), material)
# harmonic point load
function pointLoad(dpdn, xyz, J, label, t)::Float64
  dpdn[1] = t == dt ? -rho * 0.02 : 0.0
end;

Point selection

In [7]:
ICamount = 1000 # amound of initial conditions
# IDs of load elements (different initial conditions)
loadID = (collect(1 : size(fes.conn, 1)) |> shuffle)[1 : ICamount]
@show size(loadID)
# sensors are chosen as the first node of each load element.
# therefore: number of sensors = number of ICs

# random points in random instants.
# points y for evaluating solution G(u)(y).
# inputs to trunk network.
# matrix with rows [nodeID x y t]
yAmount = 1000
# IDs of chosen nodes
yNodeID = (collect(1 : size(fens.xyz, 1)) |> shuffle)[1 : yAmount]
y = hcat(
  yNodeID,
  fens.xyz[yNodeID, :],
  rand(1:nsteps, yAmount)' |> vec
)
@show size(y)
println("HDF5 file contains $(
  4 * yAmount + ICamount * yAmount
) values across $(ICamount * yAmount) samples.")

size(loadID) = (1000,)


size(y) = (1000, 4)
HDF5 file contains 1004000 values across 1000000 samples.


Solution

In [8]:
# 
data = Dict(
  # spatial-temporal points for evaluating G(u). trunk inputs
  "solEvalPoints" => y,
  # G(u)(y). Targets for DeepONet outputs.
  # each row refers to an IC case; each column, to a point y
  "solutionValues" => zeros(Float32, (ICamount, yAmount))
)
for (count, loadElementID) in enumerate(loadID)
  evalPoints = 0
  count % 100 == 0 && println("Element $count/$ICamount")
  # element to be loaded in current IC
  dipfemm = FEMMAcoustSurf(IntegDomain(subset(fes, [loadElementID]), GaussRule(2, 2)), material)
  t = 0.0 # Initial time
  # Solve the transient acoustics equations.
  # The loop executes inside the "let" local scope
  P1 = let
    P0 = deepcopy(P)
    P0.values .= 0.0 # initially all pressure is zero
    vP0 = gathersysvec(P0)
    vQ0 = zeros(eltype(vP0), size(vP0))
    # The `P1` field will be the output of this computation:
    # the final value of the pressure field
    P1 = deepcopy(P0)
    fi = ForceIntensity( # Initial load
      FCplxFlt, 1, (dpdn, xyz, J, label) -> pointLoad(dpdn, xyz, J, label, t)
    )
    La0 = distribloads(dipfemm, geom, P1, fi, 2)
    A = (2.0 / dt) * S + D + (dt / 2.0) * C
    step = 0
    while t <= tfinal
      step += 1
      t += dt
      # Update load
      fi = ForceIntensity( # Initial load
        FCplxFlt, 1, (dpdn, xyz, J, label) -> pointLoad(dpdn, xyz, J, label, t)
      )
      La1 = distribloads(dipfemm, geom, P1, fi, 2)
      # Solve for the rate of the pressure
      vQ1 = A \ ((2 / dt) * (S * vQ0) - D * vQ0 - C * (2 * vP0 + (dt / 2) * vQ0) + La0 + La1)
      # Update the value of the pressure
      vP1 = vP0 + (dt / 2) * (vQ0 + vQ1)
      # Swap variables for next step  
      vP0 = deepcopy(vP1); vQ0 = deepcopy(vQ1)
      P1 = scattersysvec!(P1, vec(vP1))
      P0 = deepcopy(P1); La0 = deepcopy(La1)
      # find rows in data["solEvalPoints"] that refer to current timestep
      yRows = findall(k -> k[4] == step, eachrow(data["solEvalPoints"]))
      evalPoints += length(yRows)
      # read pressure values in nodes of interest in current timestep, and
      # use those values to populate the row of data["solutionValues"]
      # referring to the current IC case
      data["solutionValues"][count, yRows] .= (real.(P1.values) |> vec)[Int.(data["solEvalPoints"][yRows, 1])]
    end
    P1
  end
end

Element 100/1000


Element 200/1000


Element 300/1000


Element 400/1000


Element 500/1000


Element 600/1000


Element 700/1000


Element 800/1000


Element 900/1000


Element 1000/1000

# Data

Normalize

In [9]:
standardData = copy(data)
unitData = copy(data)
# standardize
standardData["solEvalPoints"] = Float32.(copy(data["solEvalPoints"]))
standardData["solEvalPoints"][:, 2:4] = standardize(ZScoreTransform, data["solEvalPoints"][:, 2:4], dims = 1)
standardData["solutionValues"] = Float32.(standardize(ZScoreTransform, data["solutionValues"]))
# unit normalization
unitData["solEvalPoints"] = Float32.(copy(data["solEvalPoints"]))
unitData["solEvalPoints"][:, 2:4] = standardize(UnitRangeTransform, data["solEvalPoints"][:, 2:4], dims = 1)
unitData["solutionValues"] = Float32.(standardize(UnitRangeTransform, data["solutionValues"]));

Save in HDF5

In [10]:
# create files for versions of dataset: raw, standardized, and unit normalized
for (source, name) in [(data, "data") (standardData, "standardData") (unitData, "unitData")]
  h5open("./vtkFiles/$name.h5", "w") do dataFile
    for (key, value) in source
      dataFile[key] = value
    end
  end
end;

Summary statistics of file

In [11]:
for name in ["data" "standardData" "unitData"]
  println("########      $name      ########")
  h5open("./vtkFiles/$name.h5", "r") do f
    for field in HDF5.get_datasets(f)
      println("\n", HDF5.name(field)[2 : end])
      d = HDF5.read(field)
      @show size(d)
      statsum(d)
    end
  end
  println()
end;

########      data      ########



solEvalPoints


size(d) = (1000, 4)


Summary Stats:
Length:         4000
Missing Count:  0
Mean:           432.918457
Minimum:        0.000000
1st Quartile:   2.984783
Median:         6.681818
3rd Quartile:   133.000000
Maximum:        3355.000000
Standard deviation: 8.604E2
3974/4000 (99.4%) non-zero elements./n

solutionValues


size(d) = (1000, 1000)


Summary Stats:
Length:         1000000
Missing Count:  0
Mean:           -0.000000
Minimum:        -0.000011
1st Quartile:   -0.000000
Median:         -0.000000
3rd Quartile:   0.000000
Maximum:        0.000001
Standard deviation: 3.0764E-7
938531/1000000 (93.9%) non-zero elements./n

########      standardData      ########

solEvalPoints
size(d) = (1000, 4)
Summary Stats:
Length:         4000
Missing Count:  0
Mean:           414.234253
Minimum:        -1.965692
1st Quartile:   -0.546699
Median:         0.506430
3rd Quartile:   1.916054
Maximum:        3355.000000
Standard deviation: 8.6865E2
4000/4000 (100.0%) non-zero elements./n

solutionValues
size(d) = (1000, 1000)


Summary Stats:
Length:         1000000
Missing Count:  0
Mean:           0.000000
Minimum:        -24.877476
1st Quartile:   -0.206125
Median:         0.406408
3rd Quartile:   0.488069
Maximum:        3.733751
Standard deviation: 9.995E-1
1000000/1000000 (100.0%) non-zero elements./n

########      unitData      ########

solEvalPoints
size(d) = (1000, 4)
Summary Stats:
Length:         4000
Missing Count:  0
Mean:           414.599640
Minimum:        0.000000
1st Quartile:   0.333392
Median:         0.628571
3rd Quartile:   1.250000
Maximum:        3355.000000
Standard deviation: 8.6847E2
3970/4000 (99.2%) non-zero elements./n

solutionValues
size(d) = (1000, 1000)
Summary Stats:
Length:         1000000
Missing Count:  0
Mean:           0.823737
Minimum:        0.000000
1st Quartile:   0.785687
Median:         0.837336
3rd Quartile:   0.885045
Maximum:        1.000000


Standard deviation: 9.8351E-2
998998/1000000 (99.9%) non-zero elements./n

